# 🏷️ Dynamic Pricing Competition
## Exploratory Data Analysis & Solution

**Задача:** Предсказать ценовой диапазон `[price_p05, price_p95]` для товаров

**Метрика:** IoU (Intersection over Union) между истинным и предсказанным интервалами

---

## 📚 Содержание

1. [Загрузка данных и настройка](#1-загрузка-данных)
2. [Обзор данных](#2-обзор-данных)
3. [Анализ целевых переменных](#3-анализ-целевых-переменных)
4. [Временной анализ](#4-временной-анализ)
5. [Анализ категориальных признаков](#5-категориальные-признаки)
6. [Числовые признаки и погода](#6-числовые-признаки)
7. [Кластерный анализ и Cold Start](#7-кластерный-анализ)
8. [Что сработало](#8-что-сработало)
9. [Что не сработало](#9-что-не-сработало)
10. [Финальное решение](#10-финальное-решение)
11. [Выводы](#11-выводы)


In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Настройки
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.4f}'.format)

SEED = 322
np.random.seed(SEED)

# Цветовая палитра
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72', 
    'accent': '#F18F01',
    'success': '#C73E1D',
    'light': '#E8E8E8'
}

print("✅ Библиотеки загружены")


## 1. Загрузка данных <a id="1-загрузка-данных"></a>


In [ ]:
# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Преобразование дат
train['dt'] = pd.to_datetime(train['dt'])
test['dt'] = pd.to_datetime(test['dt'])

print(f"📊 Train shape: {train.shape}")
print(f"📊 Test shape:  {test.shape}")
print(f"\n📅 Train период: {train['dt'].min()} → {train['dt'].max()}")
print(f"📅 Test период:  {test['dt'].min()} → {test['dt'].max()}")


In [ ]:
# Первые строки train
train.head(10)


## 2. Обзор данных <a id="2-обзор-данных"></a>


In [ ]:
# Типы данных и пропуски
print("📋 Информация о train:")
print(train.info())
print(f"\n❓ Пропуски в train: {train.isnull().sum().sum()}")
print(f"❓ Пропуски в test:  {test.isnull().sum().sum()}")


In [ ]:
# Статистика по train
train.describe()


In [ ]:
# Анализ продуктов
train_products = set(train['product_id'].unique())
test_products = set(test['product_id'].unique())

common_products = train_products & test_products
new_products = test_products - train_products
train_only = train_products - test_products

print("🏷️ АНАЛИЗ ПРОДУКТОВ")
print("=" * 50)
print(f"Уникальных продуктов в train: {len(train_products)}")
print(f"Уникальных продуктов в test:  {len(test_products)}")
print(f"\n📊 Пересечение:")
print(f"   Общих продуктов:        {len(common_products)} ({100*len(common_products)/len(test_products):.1f}% от test)")
print(f"   Новых продуктов (test): {len(new_products)} ({100*len(new_products)/len(test_products):.1f}% от test)")
print(f"   Только в train:         {len(train_only)}")

print(f"\n⚠️ Cold Start проблема: {len(new_products)} новых продуктов в test!")


In [ ]:
# Визуализация распределения продуктов
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Количество записей на продукт в train
product_counts = train.groupby('product_id').size()
axes[0].hist(product_counts, bins=50, color=COLORS['primary'], edgecolor='white', alpha=0.8)
axes[0].axvline(product_counts.median(), color=COLORS['accent'], linestyle='--', linewidth=2, label=f'Median: {product_counts.median():.0f}')
axes[0].set_xlabel('Количество записей')
axes[0].set_ylabel('Количество продуктов')
axes[0].set_title('Распределение записей на продукт (Train)')
axes[0].legend()

# Пересечение продуктов
labels = ['Общие', 'Новые в test', 'Только в train']
sizes = [len(common_products), len(new_products), len(train_only)]
colors_pie = [COLORS['primary'], COLORS['accent'], COLORS['light']]
explode = (0, 0.1, 0)

axes[1].pie(sizes, explode=explode, labels=labels, colors=colors_pie, autopct='%1.1f%%',
            shadow=True, startangle=90)
axes[1].set_title('Пересечение продуктов Train/Test')

plt.tight_layout()
plt.show()


## 3. Анализ целевых переменных <a id="3-анализ-целевых-переменных"></a>

Целевые переменные:
- `price_p05` — нижняя граница диапазона цены (5-й квантиль)
- `price_p95` — верхняя граница диапазона цены (95-й квантиль)


In [ ]:
# Базовая статистика целевых переменных
train['width'] = train['price_p95'] - train['price_p05']
train['center'] = (train['price_p05'] + train['price_p95']) / 2
train['spread_ratio'] = train['width'] / (train['center'] + 1e-8)

print("💰 СТАТИСТИКА ЦЕЛЕВЫХ ПЕРЕМЕННЫХ")
print("=" * 50)
print(f"\nprice_p05:")
print(f"   Mean:   {train['price_p05'].mean():.4f}")
print(f"   Std:    {train['price_p05'].std():.4f}")
print(f"   Min:    {train['price_p05'].min():.4f}")
print(f"   Max:    {train['price_p05'].max():.4f}")

print(f"\nprice_p95:")
print(f"   Mean:   {train['price_p95'].mean():.4f}")
print(f"   Std:    {train['price_p95'].std():.4f}")
print(f"   Min:    {train['price_p95'].min():.4f}")
print(f"   Max:    {train['price_p95'].max():.4f}")

print(f"\nШирина интервала (p95 - p05):")
print(f"   Mean:   {train['width'].mean():.4f}")
print(f"   Std:    {train['width'].std():.4f}")
print(f"   Min:    {train['width'].min():.4f}")
print(f"   Max:    {train['width'].max():.4f}")

# Интервалы нулевой ширины
zero_width = (train['width'] < 0.001).sum()
print(f"\n⚠️ Интервалов с нулевой шириной: {zero_width} ({100*zero_width/len(train):.2f}%)")


In [ ]:
# Визуализация целевых переменных
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Распределение p05
axes[0, 0].hist(train['price_p05'], bins=50, color=COLORS['primary'], edgecolor='white', alpha=0.8)
axes[0, 0].axvline(train['price_p05'].mean(), color=COLORS['accent'], linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('price_p05')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].set_title('Распределение price_p05')

# Распределение p95
axes[0, 1].hist(train['price_p95'], bins=50, color=COLORS['secondary'], edgecolor='white', alpha=0.8)
axes[0, 1].axvline(train['price_p95'].mean(), color=COLORS['accent'], linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('price_p95')
axes[0, 1].set_ylabel('Частота')
axes[0, 1].set_title('Распределение price_p95')

# Распределение ширины
axes[0, 2].hist(train['width'], bins=50, color=COLORS['accent'], edgecolor='white', alpha=0.8)
axes[0, 2].axvline(train['width'].mean(), color=COLORS['primary'], linestyle='--', linewidth=2)
axes[0, 2].set_xlabel('Ширина интервала')
axes[0, 2].set_ylabel('Частота')
axes[0, 2].set_title('Распределение ширины (p95 - p05)')

# Scatter p05 vs p95
sample = train.sample(min(5000, len(train)), random_state=SEED)
axes[1, 0].scatter(sample['price_p05'], sample['price_p95'], alpha=0.3, c=COLORS['primary'], s=10)
axes[1, 0].plot([train['price_p05'].min(), train['price_p05'].max()], 
                [train['price_p05'].min(), train['price_p05'].max()], 'r--', linewidth=2)
axes[1, 0].set_xlabel('price_p05')
axes[1, 0].set_ylabel('price_p95')
axes[1, 0].set_title('price_p05 vs price_p95')

# Boxplot по месяцам
train.boxplot(column='width', by='month', ax=axes[1, 1])
axes[1, 1].set_xlabel('Месяц')
axes[1, 1].set_ylabel('Ширина')
axes[1, 1].set_title('Ширина интервала по месяцам')
plt.suptitle('')

# Корреляция center и width
axes[1, 2].scatter(sample['center'], sample['width'], alpha=0.3, c=COLORS['secondary'], s=10)
axes[1, 2].set_xlabel('Центр интервала')
axes[1, 2].set_ylabel('Ширина интервала')
axes[1, 2].set_title(f'Center vs Width (corr={train["center"].corr(train["width"]):.3f})')

plt.tight_layout()
plt.show()


In [ ]:
# Корреляционная матрица
target_cols = ['price_p05', 'price_p95', 'width', 'center']
corr_matrix = train[target_cols].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax, fmt='.3f')
ax.set_title('Корреляция целевых переменных')
plt.tight_layout()
plt.show()

print("\n📊 Ключевые корреляции:")
print(f"   p05 ↔ p95:     {train['price_p05'].corr(train['price_p95']):.4f}")
print(f"   center ↔ p05:  {train['center'].corr(train['price_p05']):.4f}")
print(f"   width ↔ p05:   {train['width'].corr(train['price_p05']):.4f}")


### Наблюдения по целевым переменным:

1. **Высокая корреляция** между `price_p05` и `price_p95` — можно использовать центр + ширину
2. **Ширина интервала** варьируется, но в среднем небольшая (~0.095)
3. **Есть интервалы близкие к нулевой ширине** — нужна специальная обработка
4. Данные **нормализованы** (значения около 1)


## 4. Временной анализ <a id="4-временной-анализ"></a>


In [ ]:
# Временные тренды
daily_stats = train.groupby('dt').agg({
    'price_p05': ['mean', 'std'],
    'price_p95': ['mean', 'std'],
    'width': ['mean'],
    'product_id': 'count'
}).reset_index()
daily_stats.columns = ['dt', 'p05_mean', 'p05_std', 'p95_mean', 'p95_std', 'width_mean', 'n_records']

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Средние цены по дням
axes[0].plot(daily_stats['dt'], daily_stats['p05_mean'], label='price_p05', color=COLORS['primary'], linewidth=2)
axes[0].plot(daily_stats['dt'], daily_stats['p95_mean'], label='price_p95', color=COLORS['secondary'], linewidth=2)
axes[0].fill_between(daily_stats['dt'], daily_stats['p05_mean'], daily_stats['p95_mean'], alpha=0.2, color=COLORS['accent'])
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Средняя цена')
axes[0].set_title('Временной тренд средних цен')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Ширина интервала
axes[1].plot(daily_stats['dt'], daily_stats['width_mean'], color=COLORS['accent'], linewidth=2)
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('Средняя ширина')
axes[1].set_title('Временной тренд ширины интервала')
axes[1].grid(True, alpha=0.3)

# Количество записей
axes[2].bar(daily_stats['dt'], daily_stats['n_records'], color=COLORS['primary'], alpha=0.7)
axes[2].set_xlabel('Дата')
axes[2].set_ylabel('Количество записей')
axes[2].set_title('Количество записей по дням')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Анализ по дням недели
dow_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
dow_stats = train.groupby('dow').agg({
    'price_p05': 'mean',
    'price_p95': 'mean',
    'width': 'mean'
}).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Средние цены по дням недели
x = np.arange(7)
width_bar = 0.35
axes[0].bar(x - width_bar/2, dow_stats['price_p05'], width_bar, label='price_p05', color=COLORS['primary'])
axes[0].bar(x + width_bar/2, dow_stats['price_p95'], width_bar, label='price_p95', color=COLORS['secondary'])
axes[0].set_xticks(x)
axes[0].set_xticklabels(dow_names)
axes[0].set_ylabel('Средняя цена')
axes[0].set_title('Средние цены по дням недели')
axes[0].legend()

# Ширина по дням недели
axes[1].bar(x, dow_stats['width'], color=COLORS['accent'])
axes[1].set_xticks(x)
axes[1].set_xticklabels(dow_names)
axes[1].set_ylabel('Средняя ширина')
axes[1].set_title('Средняя ширина интервала по дням недели')

plt.tight_layout()
plt.show()

print("📅 Паттерны по дням недели:")
print(f"   Максимальная ширина: {dow_names[dow_stats['width'].idxmax()]} ({dow_stats['width'].max():.4f})")
print(f"   Минимальная ширина:  {dow_names[dow_stats['width'].idxmin()]} ({dow_stats['width'].min():.4f})")


In [ ]:
# Holiday и Activity flags
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Holiday effect
holiday_stats = train.groupby('holiday_flag').agg({
    'price_p05': 'mean',
    'price_p95': 'mean',
    'width': 'mean'
}).reset_index()

x = np.arange(2)
axes[0].bar(x - 0.2, holiday_stats['price_p05'], 0.4, label='p05', color=COLORS['primary'])
axes[0].bar(x + 0.2, holiday_stats['price_p95'], 0.4, label='p95', color=COLORS['secondary'])
axes[0].set_xticks(x)
axes[0].set_xticklabels(['Обычный день', 'Праздник'])
axes[0].set_ylabel('Средняя цена')
axes[0].set_title('Влияние праздников на цены')
axes[0].legend()

# Activity effect
activity_stats = train.groupby('activity_flag').agg({
    'price_p05': 'mean',
    'price_p95': 'mean',
    'width': 'mean'
}).reset_index()

axes[1].bar(x - 0.2, activity_stats['price_p05'], 0.4, label='p05', color=COLORS['primary'])
axes[1].bar(x + 0.2, activity_stats['price_p95'], 0.4, label='p95', color=COLORS['secondary'])
axes[1].set_xticks(x)
axes[1].set_xticklabels(['Без акции', 'Акция'])
axes[1].set_ylabel('Средняя цена')
axes[1].set_title('Влияние акций на цены')
axes[1].legend()

plt.tight_layout()
plt.show()


### Временные наблюдения:

1. **Стабильный тренд** — нет резких изменений во времени
2. **Сезонность по дням недели** — небольшие различия
3. **Праздники и акции** влияют на цены — важные признаки


## 5. Анализ категориальных признаков <a id="5-категориальные-признаки"></a>


In [ ]:
# Иерархия категорий
hierarchy_cols = ['management_group_id', 'first_category_id', 'second_category_id', 'third_category_id']

print("📊 КАТЕГОРИАЛЬНАЯ ИЕРАРХИЯ")
print("=" * 50)
for col in hierarchy_cols:
    n_train = train[col].nunique()
    n_test = test[col].nunique()
    train_vals = set(train[col].unique())
    test_vals = set(test[col].unique())
    new_in_test = len(test_vals - train_vals)
    print(f"\n{col}:")
    print(f"   Train: {n_train} уникальных")
    print(f"   Test:  {n_test} уникальных")
    print(f"   Новых в test: {new_in_test}")


In [ ]:
# Распределение по management_group_id
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Количество записей
mgmt_counts = train['management_group_id'].value_counts().sort_index()
axes[0].bar(mgmt_counts.index, mgmt_counts.values, color=COLORS['primary'])
axes[0].set_xlabel('management_group_id')
axes[0].set_ylabel('Количество записей')
axes[0].set_title('Распределение по management_group_id')

# Средняя ширина по категориям
mgmt_width = train.groupby('management_group_id')['width'].mean().sort_index()
axes[1].bar(mgmt_width.index, mgmt_width.values, color=COLORS['accent'])
axes[1].set_xlabel('management_group_id')
axes[1].set_ylabel('Средняя ширина интервала')
axes[1].set_title('Средняя ширина по management_group_id')

plt.tight_layout()
plt.show()


## 6. Числовые признаки и погода <a id="6-числовые-признаки"></a>


In [ ]:
# n_stores анализ
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Распределение n_stores
axes[0].hist(train['n_stores'], bins=50, color=COLORS['primary'], edgecolor='white', alpha=0.8)
axes[0].set_xlabel('n_stores')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение n_stores')

# n_stores vs width
sample = train.sample(min(5000, len(train)), random_state=SEED)
axes[1].scatter(sample['n_stores'], sample['width'], alpha=0.3, c=COLORS['secondary'], s=10)
axes[1].set_xlabel('n_stores')
axes[1].set_ylabel('Ширина интервала')
axes[1].set_title(f'n_stores vs width (corr={train["n_stores"].corr(train["width"]):.3f})')

plt.tight_layout()
plt.show()

print(f"📊 Корреляция n_stores с целевыми:")
print(f"   n_stores ↔ p05:   {train['n_stores'].corr(train['price_p05']):.4f}")
print(f"   n_stores ↔ p95:   {train['n_stores'].corr(train['price_p95']):.4f}")
print(f"   n_stores ↔ width: {train['n_stores'].corr(train['width']):.4f}")


In [ ]:
# Погодные признаки
weather_cols = ['precpt', 'avg_temperature', 'avg_humidity', 'avg_wind_level']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(weather_cols):
    axes[i].hist(train[col], bins=40, color=COLORS['primary'], edgecolor='white', alpha=0.8)
    axes[i].axvline(train[col].mean(), color=COLORS['accent'], linestyle='--', linewidth=2)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Частота')
    axes[i].set_title(f'Распределение {col}')

plt.tight_layout()
plt.show()

print("🌤️ Корреляция погоды с шириной интервала:")
for col in weather_cols:
    corr = train[col].corr(train['width'])
    print(f"   {col}: {corr:.4f}")


## 7. Кластерный анализ и Cold Start <a id="7-кластерный-анализ"></a>


In [ ]:
# Создание признаков на уровне продукта
product_features = train.groupby('product_id').agg({
    'price_p05': ['mean', 'std'],
    'price_p95': ['mean', 'std'],
    'width': ['mean', 'std'],
    'center': ['mean'],
    'n_stores': ['mean'],
    'activity_flag': ['mean']
}).reset_index()
product_features.columns = ['product_id', 'p05_mean', 'p05_std', 'p95_mean', 'p95_std', 
                            'width_mean', 'width_std', 'center_mean', 'n_stores_mean', 'activity_mean']

# Заполнение NaN
product_features = product_features.fillna(0)

# Кластеризация
cluster_cols = ['p05_mean', 'p95_mean', 'width_mean', 'n_stores_mean', 'activity_mean']
X_cluster = product_features[cluster_cols].values
X_scaled = StandardScaler().fit_transform(X_cluster)

# KMeans
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=10)
product_features['cluster'] = kmeans.fit_predict(X_scaled)

print(f"✅ Создано {n_clusters} кластеров продуктов")
print(f"\nРаспределение по кластерам:")
print(product_features['cluster'].value_counts().sort_index())


In [ ]:
# Визуализация кластеров (PCA)
pca = PCA(n_components=2, random_state=SEED)
X_pca = pca.fit_transform(X_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Кластеры в PCA пространстве
scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=product_features['cluster'], 
                          cmap='tab10', alpha=0.6, s=30)
axes[0].set_xlabel(f'PC1 ({100*pca.explained_variance_ratio_[0]:.1f}%)')
axes[0].set_ylabel(f'PC2 ({100*pca.explained_variance_ratio_[1]:.1f}%)')
axes[0].set_title('Кластеры продуктов (PCA)')
plt.colorbar(scatter, ax=axes[0], label='Cluster')

# Характеристики кластеров
cluster_stats = product_features.groupby('cluster')[['p05_mean', 'width_mean']].mean()
axes[1].scatter(cluster_stats['p05_mean'], cluster_stats['width_mean'], 
                s=200, c=range(n_clusters), cmap='tab10', edgecolors='black')
for i in range(n_clusters):
    axes[1].annotate(str(i), (cluster_stats['p05_mean'].iloc[i], cluster_stats['width_mean'].iloc[i]),
                     fontsize=10, ha='center', va='center')
axes[1].set_xlabel('Средняя цена p05')
axes[1].set_ylabel('Средняя ширина')
axes[1].set_title('Характеристики кластеров')

plt.tight_layout()
plt.show()


In [ ]:
# Cold Start анализ
print("🧊 COLD START ANALYSIS")
print("=" * 50)

# Новые продукты в test
new_product_ids = list(new_products)
test_new = test[test['product_id'].isin(new_product_ids)]

print(f"\nНовых продуктов: {len(new_products)}")
print(f"Записей с новыми продуктами: {len(test_new)} ({100*len(test_new)/len(test):.1f}% от test)")

# Категории новых продуктов
print(f"\nКатегории новых продуктов:")
for col in hierarchy_cols:
    common_cats = set(train[col].unique()) & set(test_new[col].unique())
    print(f"   {col}: {len(common_cats)} общих категорий из {test_new[col].nunique()}")


## 8. Что сработало <a id="8-что-сработало"></a>

### ✅ Методы, которые улучшили результат:

### 1. Feature Engineering на уровне продукта

**Агрегированные статистики по продукту:**
- Среднее, std, min, max, median для `price_p05` и `price_p95`
- Взвешенное среднее с временным затуханием (более свежие данные важнее)
- Target encoding с Bayesian smoothing

**Почему работает:** Цены продукта достаточно стабильны, исторические значения — лучший предиктор

### 2. Кластеризация продуктов

**Подход:**
- KMeans на агрегированных признаках продуктов
- Статистики кластера как дополнительные признаки
- Fallback на кластерные средние для холодного старта

**Почему работает:** Похожие продукты имеют схожие ценовые паттерны

### 3. Иерархические категориальные признаки

**Подход:**
- Target encoding для всех уровней категорий
- Fallback по иерархии: `third_category` → `second_category` → `first_category` → `global`

**Почему работает:** Помогает для новых продуктов и добавляет контекст

### 4. Ансамбль MAE + Quantile моделей

**Подход:**
- MAE loss для базового предсказания
- Quantile loss (0.05 и 0.95) для границ
- Оптимальное взвешивание через grid search на валидации

**Почему работает:** Quantile дает правильную форму интервала, MAE — стабильный центр

### 5. Пост-калибровка на IoU

**Подход:**
- Линейная трансформация: `pred = pred * scale + shift`
- Оптимизация L-BFGS-B напрямую по IoU метрике
- Отдельная калибровка для p05 и p95

**Почему работает:** Корректирует систематические смещения модели

### 6. CatBoost с правильными гиперпараметрами

**Ключевые параметры:**
- `depth=7` — достаточная глубина без переобучения
- `learning_rate=0.025` — медленное обучение, лучше обобщение
- `iterations=3500` с early stopping
- `bagging_temperature=0.3` — небольшая регуляризация

**Почему работает:** CatBoost хорошо работает с категориальными фичами и требует меньше тюнинга


## 9. Что не сработало <a id="9-что-не-сработало"></a>

### ❌ Методы, которые НЕ улучшили результат:

### 1. Предсказание центра + ширины вместо p05 + p95

**Пробовали:** 
- Отдельные модели для `center = (p05 + p95) / 2` и `width = p95 - p05`
- Реконструкция: `p05 = center - width/2`, `p95 = center + width/2`

**Почему не сработало:**
- Ошибки в width и center накапливаются
- Прямое предсказание p05/p95 более стабильно для IoU

### 2. Сложные временные признаки

**Пробовали:**
- Лаговые признаки (lag_1, lag_7, lag_14)
- Скользящие средние (rolling_7, rolling_14)
- Тренды и сезонность

**Почему не сработало:**
- Нет временной последовательности в test (даты не продолжают train)
- Агрегированные статистики по продукту работают лучше

### 3. Нейросетевые подходы

**Пробовали:**
- Простой MLP
- TabNet
- Entity embeddings для категорий

**Почему не сработало:**
- Малый объем данных (~30K записей)
- Gradient boosting лучше на табличных данных такого размера

### 4. Сложные ансамбли

**Пробовали:**
- CatBoost + LightGBM + XGBoost
- Stacking с метамоделью

**Почему не сработало:**
- Модели слишком коррелированы
- Один хорошо настроенный CatBoost работает не хуже

### 5. Погодные признаки как основные предикторы

**Пробовали:**
- Создание сложных погодных индексов
- Интеракции погода × категория

**Почему не сработало:**
- Слабая корреляция с целевыми переменными
- Добавляют шум, но не сигнал

### 6. Outlier removal

**Пробовали:**
- Isolation Forest
- Z-score filtering

**Почему не сработало:**
- Данные уже чистые и нормализованные
- Потеря информации без улучшения метрики


## 10. Финальное решение <a id="10-финальное-решение"></a>


In [ ]:
# IoU метрика
def calculate_iou(y_true_p05, y_true_p95, y_pred_p05, y_pred_p95, epsilon=1e-6):
    """Mean IoU для ценовых интервалов"""
    width_true = np.maximum(y_true_p95 - y_true_p05, epsilon)
    width_pred = np.maximum(y_pred_p95 - y_pred_p05, epsilon)
    
    intersection_lower = np.maximum(y_true_p05, y_pred_p05)
    intersection_upper = np.minimum(y_true_p95, y_pred_p95)
    intersection = np.maximum(0, intersection_upper - intersection_lower)
    
    union = width_true + width_pred - intersection
    union = np.maximum(union, epsilon)
    
    iou = intersection / union
    return float(np.mean(iou))

print("✅ IoU метрика определена")


### Архитектура решения

```
┌─────────────────────────────────────────────────────────────────┐
│                      Feature Engineering                        │
├─────────────────────────────────────────────────────────────────┤
│  • Агрегации по продукту (mean, std, last, weighted)           │
│  • Target encoding для категорий                                │
│  • Кластеры продуктов (KMeans)                                  │
│  • Календарные признаки (циклические)                           │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    5-Fold Cross-Validation                      │
├─────────────────────────────────────────────────────────────────┤
│  Для каждого фолда:                                             │
│  ┌──────────────┐    ┌──────────────┐                          │
│  │ CatBoost MAE │    │CatBoost MAE  │                          │
│  │   (p05)      │    │   (p95)      │                          │
│  └──────────────┘    └──────────────┘                          │
│  ┌──────────────┐    ┌──────────────┐                          │
│  │CatBoost Q05  │    │CatBoost Q95  │                          │
│  │  (Quantile)  │    │  (Quantile)  │                          │
│  └──────────────┘    └──────────────┘                          │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Optimal Blending                             │
├─────────────────────────────────────────────────────────────────┤
│  p05 = w05 * MAE_p05 + (1-w05) * Quantile_p05                  │
│  p95 = w95 * MAE_p95 + (1-w95) * Quantile_p95                  │
│  (веса подбираются grid search на валидации)                    │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Calibration                                  │
├─────────────────────────────────────────────────────────────────┤
│  p05_cal = p05 * scale + shift_p05                              │
│  p95_cal = p95 * scale + shift_p95                              │
│  (оптимизация L-BFGS-B по IoU)                                  │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Ensemble Average                             │
├─────────────────────────────────────────────────────────────────┤
│  final_p05 = mean(p05_fold1, ..., p05_fold5)                    │
│  final_p95 = mean(p95_fold1, ..., p95_fold5)                    │
└─────────────────────────────────────────────────────────────────┘
```


In [ ]:
# Ключевые гиперпараметры CatBoost
catboost_params = {
    'iterations': 3500,
    'learning_rate': 0.025,
    'depth': 7,
    'l2_leaf_reg': 4.0,
    'min_data_in_leaf': 10,
    'bagging_temperature': 0.3,
    'random_strength': 2.0,
    'border_count': 128,
    'grow_policy': 'SymmetricTree',
    'bootstrap_type': 'Bayesian',
    'loss_function': 'MAE',  # или Quantile:alpha=0.05/0.95
    'early_stopping_rounds': 300,
    'random_seed': 322,
}

print("📋 Ключевые параметры модели:")
for k, v in catboost_params.items():
    print(f"   {k}: {v}")


## 11. Выводы <a id="11-выводы"></a>

### 📊 Ключевые выводы из EDA:

1. **Данные нормализованы** — значения цен около 1, это упрощает обработку

2. **Высокая корреляция p05 и p95** — прямое предсказание обеих границ работает лучше, чем center+width

3. **Cold Start проблема** — ~23% записей в test относятся к новым продуктам, требуется fallback через категории и кластеры

4. **Исторические данные продукта** — лучший предиктор цены, важнее всех других признаков

5. **Погода и флаги** — слабое влияние, но полезны как дополнительные признаки

### 🎯 Итоговый подход:

- **Feature Engineering**: агрегации по продукту + target encoding категорий + кластеры
- **Модель**: CatBoost с MAE и Quantile loss, ансамбль 5 фолдов
- **Калибровка**: линейная трансформация с оптимизацией по IoU
- **Cold Start**: fallback по иерархии категорий и кластерным средним


In [ ]:
print("\n" + "="*60)
print("         DYNAMIC PRICING COMPETITION - SUMMARY")
print("="*60)
print(f"\n📊 Dataset: {len(train)} train / {len(test)} test records")
print(f"🏷️ Products: {len(train_products)} train / {len(test_products)} test")
print(f"🧊 Cold start: {len(new_products)} new products ({100*len(new_products)/len(test_products):.1f}%)")
print(f"\n🎯 Metric: IoU (Intersection over Union)")
print(f"📈 Target: price_p05 (5th percentile), price_p95 (95th percentile)")
print(f"\n✅ Best approach: CatBoost + Product aggregations + Calibration")
print("="*60)
